In [1]:
import sys
sys.path.append('..')
import os
from tqdm import tqdm
import pandas as pd
import warnings
from src.lesion import perform_lesion_experiment, do_lesion_hypo_tests
from src.pointers import DATA_PATHS
from src.experiment_tagging import get_model_path

Using TensorFlow backend.


In [2]:
models = ['CNN-VGG-CIFAR10', 'CNN-VGG-CIFAR10+DROPOUT+L2REG'
          # 'CNN-VGG-CIFAR10+L1REG', 'CNN-VGG-CIFAR10+L2REG',
          # 'CNN-VGG-CIFAR10+DROPOUT', , 'CNN-VGG-CIFAR10+MOD-INIT'
          ]

n_clust = 12
n_shuffles = 20
n_workers = 5
n_reps = 5
is_unpruned = True
dataset_name = 'cifar10_full'
os.environ['CUDA_VISIBLE_DEVICES'] = '1,3'

In [3]:
all_results = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    for tag in tqdm(models):

        paths = get_model_path(tag, filter_='all')[-n_reps:]

        for path in paths:

            (true_results,
             all_random_results,
             metadata,
             evaluation) = perform_lesion_experiment('.' + DATA_PATHS[dataset_name],
                                                     path, n_clusters=n_clust,
                                                     n_shuffles=n_shuffles,
                                                     unpruned=is_unpruned,
                                                     depth=3,
                                                     n_side=32)
            hypo_results = do_lesion_hypo_tests(evaluation, true_results, all_random_results)
            chi2_p_means = hypo_results['chi2_p_means']
            chi2_p_ranges = hypo_results['chi2_p_ranges']
            combined_p_means = hypo_results['combined_p_means']
            combined_p_ranges = hypo_results['combined_p_ranges']
            effect_means = hypo_results['effect_factor_means']
            effect_ranges = hypo_results['effect_factor_range']

            model_results = {'is_unpruned': is_unpruned, 'model_tag': tag,
                             'dataset': dataset_name, 'chi2_p_means': chi2_p_means,
                             'chi2_p_ranges': chi2_p_ranges, 'combined_p_means': combined_p_means,
                             'effect_ranges': effect_ranges, 'effect_means': effect_means,
                             'combined_p_ranges': combined_p_ranges}
            all_results.append(pd.Series(model_results))

result_df = pd.DataFrame(all_results)
savepath = '../results/lesion_results_cnn_vgg.csv'
result_df.to_csv(savepath)
result_df


  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [1:26:46<8:40:38, 5206.45s/it]

 29%|██▊       | 2/7 [2:52:34<7:12:24, 5188.83s/it]

 43%|████▎     | 3/7 [3:53:28<5:15:14, 4728.53s/it]

 57%|█████▋    | 4/7 [8:10:36<6:36:54, 7938.21s/it]

 71%|███████▏  | 5/7 [14:18:05<6:45:43, 12171.61s/it]

 86%|████████▌ | 6/7 [20:34:54<4:15:02, 15302.65s/it]

100%|██████████| 7/7 [21:52:42<00:00, 12112.36s/it]  

100%|██████████| 7/7 [21:52:42<00:00, 11251.78s/it]

,is_unpruned,model_tag,network,dataset,chi2_p_means,chi2_p_stds,combined_p_means,combined_p_stds
0,False,MNIST,MLP,mnist,1.362690e-20,1.217738e-01,3.436946e-09,8.425660e-01
1,False,MNIST,MLP,mnist,6.569656e-07,5.160085e-02,1.241211e-05,8.630156e-01
2,False,MNIST,MLP,mnist,1.133536e-14,7.853003e-07,3.308756e-07,5.022651e-01
3,False,MNIST,MLP,mnist,1.047182e-13,1.002984e-02,3.284628e-09,9.939614e-01
4,False,MNIST,MLP,mnist,8.391256e-24,2.443453e-05,9.473283e-08,7.941233e-01
5,False,CIFAR10,MLP,cifar10,2.560037e-06,8.638070e-14,7.879100e-04,2.156128e-04
6,False,CIFAR10,MLP,cifar10,4.018413e-10,9.516042e-07,5.360029e-05,6.479042e-01
7,False,CIFAR10,MLP,cifar10,1.152081e-10,9.981256e-10,3.392675e-05,2.438592e-01
8,False,CIFAR10,MLP,cifar10,5.154715e-08,2.327078e-02,1.441507e-04,9.100498e-02
9,False,CIFAR10,MLP,cifar10,2.214766e-04,8.127249e-05,5.106075e-03,1.424048e-02
